Household Survey Converer

Data was obtained at 

https://www12.statcan.gc.ca/nhs-enm/2011/dp-pd/prof/details/download-telecharger/comprehensive/comp-csv-tab-nhs-enm.cfm?Lang=E

This code will convert that spreadsheet into useful data for Syntheco, and eventually into Neurohub as a standard bit of information.

In [1]:
import csv
import os
import random
import math
import pandas as pd
import numpy as np
import itertools

Utility Functions

random_round_int - converts a floating point number to an integer where a random up or down based on 50%
fill_counts_uniform - takes an array of integers and a total and fills in the -1s with a uniform distribution

In [2]:
def random_round_int(float_num):
    rand_prob = random.random()
    if rand_prob <= .50:
        return int(math.floor(float_num))
    else:
        return int(math.ceil(float_num))

In [3]:
def fill_counts_uniform(counts,total):
    denom = 0
    total_uni = total
    counts_new = []
    for x in counts:
        if x == -1:
            denom += 1
        else:
            total_uni = total_uni - x
    
    #print(denom)
    #print(total_uni)
    if denom == 0:
        return counts
    
    uni_value = float(total_uni)/float(denom)
    if uni_value < 0:
        uni_value = 0
    #print(uni_value)
    
    counts_new = []
    for x in counts:
        if x < 0:
            counts_new.append(random_round_int(uni_value))
        else:
            counts_new.append(x)
    
    return counts_new

Here are input variable definitions

In [4]:
data_location = "C:/Users/shawn/Programs/Syntheco_darcy/Data"
quebec_datatable = os.path.join(data_location,"99-004-XWE2011001-401-QUE.csv")
output_marginal_table = os.path.join(data_location,"HHInc_Marginal_QUE.csv")
output_marginal_key_file = os.path.join(data_location,"HHInc_Marginal_QUE_key.csv")

Now begins the main section of the tool.

In [5]:
data = pd.read_csv(quebec_datatable, encoding='cp1252',dtype={'Geo_Code':str,'Flag Total':str})

C:\Users\shawn\Anaconda3\envs\syntheco\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


This now uses only pandas to do the filtering, TODO: add the filling in of values like before



In [6]:
def gather_counts_from_topic_metric(data_, desired_topic_, desired_metric_,tract_mask_,n_):
    is_topic = data_['Topic'] == desired_topic_
    is_que = data_['Geo_Code'].str.startswith(str(tract_mask_))
    df = data_[is_topic & is_que]
    
    idx = df.index.get_indexer_for(df[df.Characteristic==desired_metric_].index)
    idx = np.asarray(list(itertools.chain.from_iterable([list(range(x,x+n_)) for x in idx])))
    df = df.iloc[idx]
    
    # Convert the categories to an index paramter
    cats = df["Characteristic"].head(14).tolist()
    for c in cats:
        df = df.replace(c,cats.index(c))
    return {"Categories":cats,"Data":df}

In [7]:
df_data = gather_counts_from_topic_metric(data,
                                     "Income of households in 2010",
                                     "Household total income in 2010 of private households",
                                     "462",14)

with open("HHInc2011_marginals.csv","w",newline='') as f:
    df_data['Data'].pivot_table('Total',['Geo_Code'],'Characteristic').to_csv(f,quoting=csv.QUOTE_ALL)

with open("HHInc2011_metadata.csv","w",newline='') as f:
    csv_write = csv.writer(f)
    csv_write.writerow(["index","value"])
    for x in df_data['Categories']:
        csv_write.writerow([df_data['Categories'].index(x),x.strip()])

NameError: name 'desired_metric' is not defined